In [9]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
from keras.layers import Add
from keras.layers import Input
from keras.models import Model
from keras.layers import Flatten

In [35]:
from keras import backend as K

In [57]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger,EarlyStopping,ModelCheckpoint

In [58]:
x_train = np.load("x_train_out.npy")
y_train = np.load("y_train_out.npy")
x_test = np.load("x_test_out.npy")
y_test = np.load("y_test_out.npy")

In [59]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [60]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [61]:
#one hot encoding of target labels
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [15]:
""""
Initial Convolutional layer which is common to all ResNet models.
""""
#Input : Input tensor
#filter : No of Convolutional filters
#stride : stride length 
#kernel_size : Convolutional filter size

#NOTE : kernel size and stride length are 7 and 2 in resnet paper

# Kernel size of 3 and stride length of 2 and 1 are tried for CIFAR-10 dataset because of low resolution of the images

def initial_conv(Input, filters, stride = 1,kernel_size = 7):
    
    x = Conv2D(filters, kernel_size=(kernel_size,kernel_size), strides = (stride,stride), padding = "same")(Input)
    
    x = BatchNormalization()(x)
    
    x = Activation('relu')(x)
    return x

In [20]:
""""
Residual Block with projection shortcut to match the dimensions using 1*1 convolutions.

Note : This is basic residual Block, here all the convolutions are of same size and the depth is kept constant

""""

# Input : Input tensor
# filters : No of Filters
# Stride : stride length 
# Note : Stride 2 is used to downsample the image in CONV2,CONV3 and CONV4 blocks
# Dropout : Adds Dropout layer if dropout is greater than 0

def expand_conv_basic_block(Input, filters, stride=1, dropout = 0.0):
    Init = Input
    
    #First conv which is used to downsample the image
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #Optional Dropout layer
    if(dropout > 0.0):
        x = Dropout(dropout)(x)
    
    x = Conv2D(filters,kernel_size=(3,3),strides = (1,1),padding = "same")(x)
    x = BatchNormalization()(x)
    
    #Projection shortcut to make skip connection(Paper terminology)
    skip_conv = Conv2D(filters, kernel_size = (1,1),strides = (stride,stride),padding = "same")(Input)
    skip = BatchNormalization()(skip_conv)
    
    #Skip connection
    x = Add()([x,skip])
    return x

In [21]:
"""
Residual networks with basic Identity shortcuts

"""
# Input : Input tensor
# filters : No of Filters
# Stride : stride length 
# Dropout : Adds Dropout layer if dropout is greater than 0

def normal_conv_basic_block(Input, filters, stride = 1, dropout = 0.0):
    
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #Optional Dropout layer
    if(dropout > 0.0):
        x = Dropout(dropout)(x)
    
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(x)
    x = BatchNormalization()(x)
    
    #Identity skip connection
    x = Add()([x,Input])
    
    return x

In [51]:
""""
Residual Block with projection shortcut to match the dimensions using 1*1 convolutions.

Note : This is bottleneck residual block. Here first 1*1 convolution is used to reduce depth, followed by 3*3 
        and last 1*1 is used to restore the depth

""""

# Input : Input tensor
# filters : No of Filters
# Stride : stride length 
# Note : Stride 2 is used to downsample the image in CONV2,CONV3 and CONV4 blocks
# Dropout : Adds Dropout layer if dropout is greater than 0

def expand_conv_bottleneck_block(Input,filters,stride=1,dropout = 0.0):
    
    #Contracting 1*1 conv
    x = Conv2D(filters,kernel_size=(1,1),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #if(dropout > 0.0):
     #   x = Dropout(dropout)(x)
    
    #Depth preserving 3*3 conv
    x = Conv2D(filters,kernel_size=(3,3),strides = (1,1),padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #if(Dropout > 0.0):
     #   x = Dropout(dropout)(x)
    
    #Expanding 1*1 Conv
    x = Conv2D(filters*4,kernel_size=(1,1),strides = (1,1),padding = "same")(x)
    x = BatchNormalization()(x)
    
    #Projection shortcut
    skip_conv = Conv2D(filters*4,kernel_size = (1,1), strides = (stride, stride),padding = "same")(Input)
    skip = BatchNormalization()(skip_conv)
    
    #Skip connection
    x = Add()([x,skip])
    
    return x

In [52]:
"""
Residual networks with bottleneck Identity shortcuts

"""
# Input : Input tensor
# filters : No of Filters
# Stride : stride length 
# Dropout : Adds Dropout layer if dropout is greater than 0


def normal_conv_bottleneck_block(Input, filters, stride = 1, dropout = 0.0):
    #Contracting 1*1 conv
    x = Conv2D(filters,kernel_size=(1,1),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #if(dropout > 0.0):
     #   x = Dropout(dropout)(x)
        
    #Depth preserving 3*3 Conv
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
   # if(Dropout > 0.0):
    #    x = Dropout(dropout)(x)
    
    #Expanding 1*1 Conv
    x = Conv2D(filters*4,kernel_size=(1,1),strides = (stride,stride),padding = "same")(x)
    x = BatchNormalization()(x)
    
    #Identity skip connection
    x = Add()([x,Input])
    
    return x

In [53]:
"""
Helper function to Build ResNet using basic residual blocks.
Used when the total no of layers are less than 50.

"""
#h = height of the image
#w = width of the image
#no_of_outputs = no of classification classes
#r1 = No of times first conv block should be repeated
#r2 = No of times second conv block should be repeated
#r3 = No of times third conv block should be repeated
#r4 = No of times fourth conv block should be repeated

# first_conv_stride = stride which will be used for initial conv block
# first_max_pool = boolean to decide to apply max pooling or not
# first_conv_size = kernel size which will be used for initial conv block

#NOTE : The above three parameters are used only for cifar 10 data set coz of it's low resolution. 
        #For ImageNet Dataset they can be left as default


def build_basic_resnet(h, w, no_of_outputs, r1,r2,r3,r4, first_conv_stride = 2, first_max_pool = True,first_conv_kernel_size = 7):
    
    #Creating input tensor
    inputs = Input(shape = (h,w,3), name = "image_input")
    
    # Inital Conv block
    x = initial_conv(inputs,64,first_conv_stride,first_conv_kernel_size)
    
    #Optional Max pooling layer
    if(first_max_pool):
        x = MaxPooling2D(pool_size=(2,2))(x)
    

    #Expanding block1 with projection shortcut
    x = expand_conv_basic_block(x,64,1)
    x = Activation('relu')(x)
    
    #Repeating block of Conv1
    for i in range(r1-1):
        x = normal_conv_basic_block(x,64)
        x = Activation('relu')(x)
    
    #Expanding block2 with projection shortcut
    x = expand_conv_basic_block(x,128,2)
    x = Activation('relu')(x)
    
    #Repeating block of Conv2
    for i in range(r2-1):
        x = normal_conv_basic_block(x,128)
        x = Activation('relu')(x)
    
    #Expanding block3 with projection shortcut
    x = expand_conv_basic_block(x,256,2)
    x = Activation('relu')(x)
    
    #Repeating block of Conv3
    for i in range(r3-1):
        x = normal_conv_basic_block(x,256)
        x = Activation('relu')(x)
          
     #Expanding block4 with projection shortcut
    x = expand_conv_basic_block(x,512,2)
    x = Activation('relu')(x)
    
    #Repeating block of Conv3
    for i in range(r4-1):
        x = normal_conv_basic_block(x,512)
        x = Activation('relu')(x)
    
    shape = K.int_shape(x)
    
    #Average pooling layer
    x = AveragePooling2D(pool_size=(shape[1], shape[2]),
                                 strides=(1, 1))(x)
   # x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    
    #Classifier Block
    x = Dense(no_of_outputs,activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
    

In [40]:
model = build_basic_resnet(32,32,10,2,2,2,2,1,False,7)

In [41]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_158 (Conv2D)              (None, 32, 32, 64)    9472        image_input[0][0]                
____________________________________________________________________________________________________
batch_normalization_158 (BatchNo (None, 32, 32, 64)    256         conv2d_158[0][0]                 
____________________________________________________________________________________________________
activation_127 (Activation)      (None, 32, 32, 64)    0           batch_normalization_158[0][0]    
___________________________________________________________________________________________

In [42]:
from keras.utils import plot_model 

In [43]:
plot_model(model,"Resnet.png",show_shapes=True)

In [54]:
"""
Helper function to Build ResNet using bottleneck residual blocks.
Used when the total no of layers are more than 50.

"""
#h = height of the image
#w = width of the image
#no_of_outputs = no of classification classes
#r1 = No of times first conv block should be repeated
#r2 = No of times second conv block should be repeated
#r3 = No of times third conv block should be repeated
#r4 = No of times fourth conv block should be repeated

# first_conv_stride = stride which will be used for initial conv block
# first_max_pool = boolean to decide to apply max pooling or not
# first_conv_size = kernel size which will be used for initial conv block

#NOTE : The above three parameters are used only for cifar 10 data set coz of it's low resolution. 
        #For ImageNet Dataset they can be left as default


def build_bottleneck_resnet(h, w, no_of_outputs, r1,r2,r3,r4, first_conv_stride = 2, first_max_pool = True,first_conv_kernel_size = 7):
    
    #Creating input tensor
    inputs = Input(shape = (h,w,3), name = "image_input")
    
    # Inital Conv block
    x = initial_conv(inputs,64,first_conv_stride,first_conv_kernel_size)
    
    #Optional Max pooling layer
    if(first_max_pool):
        x = MaxPooling2D(pool_size=(2,2))(x)
        
    #Expanding block1 with projection shortcut
    x = expand_conv_bottleneck_block(x,64,1)
    x = Activation('relu')(x)
    
    #Repeating block of Conv1
    for i in range(r1-1):
        x = normal_conv_bottleneck_block(x,64)
        x = Activation('relu')(x)
    
    #Expanding block2 with projection shortcut
    x = expand_conv_bottleneck_block(x,128,2)
    x = Activation('relu')(x)
    
    #Repeating block of Conv2
    for i in range(r2-1):
        x = normal_conv_bottleneck_block(x,128)
        x = Activation('relu')(x)
    
    #Expanding block3 with projection shortcut
    x = expand_conv_bottleneck_block(x,256,2)
    x = Activation('relu')(x)
    
    #Repeating block of Conv3
    for i in range(r3-1):
        x = normal_conv_bottleneck_block(x,256)
        x = Activation('relu')(x)
    
    #Expanding block4 with projection shortcut
    x = expand_conv_bottleneck_block(x,512,2)
    x = Activation('relu')(x)
    
    #Repeating block of Conv4
    for i in range(r4-1):
        x = normal_conv_bottleneck_block(x,512)
        x = Activation('relu')(x)
    
    shape = K.int_shape(x)
    
    #Average pooling layer
    x = AveragePooling2D(pool_size=(shape[1], shape[2]),
                                 strides=(1, 1))(x)
   # x = GlobalAveragePooling2D()(x)

    #Classifier Block
    x = Flatten()(x)
    x = Dense(no_of_outputs,activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
    

In [55]:
model = build_bottleneck_resnet(32,32,10,3,4,6,3,1,False,7)

In [56]:
plot_model(model,"Resnet_bottleneck.png",show_shapes=True)

In [50]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_208 (Conv2D)              (None, 32, 32, 64)    9472        image_input[0][0]                
____________________________________________________________________________________________________
batch_normalization_208 (BatchNo (None, 32, 32, 64)    256         conv2d_208[0][0]                 
____________________________________________________________________________________________________
activation_169 (Activation)      (None, 32, 32, 64)    0           batch_normalization_208[0][0]    
___________________________________________________________________________________________

In [62]:
model.compile(loss='categorical_crossentropy',
        optimizer="Adam",
        metrics=['accuracy'])

In [63]:
#Defining Callback functions which will be called by model during runtime when specified condition satisfies

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('ResNet50_without_dropout_without_conv_without_pool.csv')
early_stopper = EarlyStopping(min_delta=0.001, patience=30)
model_chekpoint = ModelCheckpoint("ResNet50_without_dropout_without_conv_without_pool.hdf5",monitor = 'val_loss',verbose = 1,save_best_only=True)

In [64]:
#model Parameters
batch_size = 64
data_augmentation = True
epochs = 100

In [ ]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,verbose=1,validation_data=(x_test,y_test),callbacks = [lr_reducer,early_stopper,csv_logger,model_chekpoint])
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=batch_size*4,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    
    
    